# Hypothesis Testing – Economic Stress and Digital Behavior

This notebook implements the hypothesis testing part of the project:

> **Economic Stress and Digital Behavior: How Economic Changes Shape Social Media Usage in Europe**

We use two processed panel datasets:

- `panel_annual.csv` / `panel_annual_final.csv` – country × year panel with:
  - `sm_participation` – Eurostat social-media participation (% of individuals)
  - `inflation`, `unemployment` – annual macro indicators
  - `cci` – consumer confidence index (annual average)
  - `stress` – combined economic stress index (z-score of inflation and unemployment minus z-score of confidence)
  - `d_sm` – year-to-year change in social-media participation (within country)
- `panel_monthly_h3.csv` – country × year × month panel for H3 with:
  - unemployment rate, HICP index
  - platform shares (Facebook, Instagram, YouTube, LinkedIn)
  - `ent_share`, `prof_share` – entertainment vs professional categories
  - monthly changes `d_unemp`, `d_infl`, `d_ent`, `d_prof`

We test the following hypotheses:

- **H1:** Higher economic stress is associated with higher social-media participation.
- **H2:** When consumer confidence is low, the effect of economic stress on social-media participation is stronger.
- **H3:** During periods of economic stress, platform use shifts:
    - rising unemployment is associated with increases in professional platforms (e.g., LinkedIn),
    - rising inflation is associated with increases in entertainment platforms (e.g., Facebook, Instagram, YouTube).

In [1]:
# ------------------------------------------------------------
# Imports and paths
# ------------------------------------------------------------
import pandas as pd
import numpy as np
from pathlib import Path
import scipy.stats as st
import statsmodels.formula.api as smf

# Project paths
PROJECT_ROOT = Path.home() / "Desktop" / "DSA210"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

print("Using processed data from:", DATA_PROCESSED)

# ------------------------------------------------------------
# Load annual panel (panel_a)
# ------------------------------------------------------------
annual_candidates = ["panel_annual_final.csv", "panel_annual.csv"]
panel_a = None
for fname in annual_candidates:
    path = DATA_PROCESSED / fname
    if path.exists():
        print("Loaded annual panel from:", fname)
        panel_a = pd.read_csv(path)
        break

if panel_a is None:
    raise FileNotFoundError("No annual panel file found. Expected one of: " + ", ".join(annual_candidates))

# ------------------------------------------------------------
# Load monthly panel for H3 (panel_m)
# ------------------------------------------------------------
monthly_path = DATA_PROCESSED / "panel_monthly_h3.csv"
if not monthly_path.exists():
    raise FileNotFoundError("Monthly panel 'panel_monthly_h3.csv' not found in data/processed/.")

panel_m = pd.read_csv(monthly_path)
print("Monthly panel shape:", panel_m.shape)

display(panel_a.head())
display(panel_m.head())

Using processed data from: /Users/ibrahimgozlukaya/Desktop/DSA210/data/processed
Loaded annual panel from: panel_annual_final.csv
Monthly panel shape: (306593, 15)


,geo,year,sm_participation,inflation,unemployment,cci,Facebook,Instagram,YouTube,LinkedIn,ent_share,prof_share,ent_minus_prof,d_sm,stress
0,AL,2018,48.33,NaN,NaN,NaN,79.240833,3.094167,7.065000,0.298333,89.400000,0.298333,89.101667,NaN,NaN
1,AL,2019,52.13,NaN,NaN,NaN,69.003333,5.556667,4.623333,0.275833,79.183333,0.275833,78.907500,3.80,NaN
2,AL,2020,54.80,NaN,NaN,NaN,75.620000,6.143333,3.115000,0.157500,84.878333,0.157500,84.720833,2.67,NaN
3,AL,2021,60.77,NaN,NaN,NaN,91.660833,1.360000,1.035833,0.115833,94.056667,0.115833,93.940833,5.97,NaN
4,AL,2022,64.07,NaN,NaN,NaN,93.218333,1.830000,1.087500,0.140000,96.135833,0.140000,95.995833,3.30,NaN


,geo,year,month,facebook,instagram,youtube,linkedin,unemployment,hicp_index,ent_share,prof_share,d_unemp,d_infl,d_ent,d_prof
0,AL,2010,10,86.40,0.0,11.36,0.03,NaN,NaN,32.586667,0.03,NaN,NaN,NaN,NaN
1,AL,2010,11,87.67,0.0,10.63,0.03,NaN,NaN,32.766667,0.03,NaN,NaN,0.180000,0.00
2,AL,2010,12,87.53,0.0,10.80,0.04,NaN,NaN,32.776667,0.04,NaN,NaN,0.010000,0.01
3,AL,2011,1,90.20,0.0,8.28,0.05,NaN,NaN,32.826667,0.05,NaN,NaN,0.050000,0.01
4,AL,2011,2,88.35,0.0,9.84,0.05,NaN,NaN,32.730000,0.05,NaN,NaN,-0.096667,0.00


## H1 – Economic Stress and Social-Media Participation

**Hypothesis**

- **H1:** Years with higher economic stress have higher social-media participation.
- **H0:** There is no systematic difference in participation between low-stress and high-stress years.

We use the combined **economic stress index** (`stress`) constructed from inflation, unemployment, and (negatively) consumer confidence.

Two complementary approaches are used:

1. **Group comparison (Welch two-sample t-test)**  
   Compare mean `sm_participation` between:
   - low-stress years (bottom 50% of the stress distribution), and  
   - high-stress years (top 50%).

2. **Linear association (correlation with participation change)**  
   Check whether the stress index is correlated with year-to-year change in participation `d_sm` within countries.

In [2]:
# ------------------------------------------------------------
# H1 – Welch t-test: low vs high stress years
# ------------------------------------------------------------

# Keep observations with non-missing stress and participation
df_h1 = panel_a.dropna(subset=["stress", "sm_participation"]).copy()

# Median split (or q=2) into low vs high stress
df_h1["stress_group"] = pd.qcut(df_h1["stress"], 2, labels=["Low stress", "High stress"])

low  = df_h1[df_h1["stress_group"] == "Low stress"]["sm_participation"]
high = df_h1[df_h1["stress_group"] == "High stress"]["sm_participation"]

tstat, pval = st.ttest_ind(high, low, equal_var=False)

print("H1 – Welch t-test (High vs Low stress)")
print(f"t-statistic = {tstat:.3f}")
print(f"p-value     = {pval:.4f}")
print(f"Mean SM participation – Low stress  : {low.mean():.2f}")
print(f"Mean SM participation – High stress : {high.mean():.2f}")
print(f"N_low = {len(low)}, N_high = {len(high)}")

# ------------------------------------------------------------
# H1 – correlation between stress and ΔSM (within countries)
# ------------------------------------------------------------

# Make sure year-to-year change is available (or recompute)
if "d_sm" not in panel_a.columns:
    panel_a = panel_a.sort_values(["geo", "year"])
    panel_a["d_sm"] = panel_a.groupby("geo")["sm_participation"].diff()

df_corr = panel_a.dropna(subset=["stress", "d_sm"]).copy()
r_h1, p_h1 = st.pearsonr(df_corr["stress"], df_corr["d_sm"])

print("\nH1 – Pearson correlation between stress and ΔSM (yearly change)")
print(f"r = {r_h1:.3f}, p = {p_h1:.4f}, N = {len(df_corr)}")

H1 – Welch t-test (High vs Low stress)
t-statistic = -2.039
p-value     = 0.0425
Mean SM participation – Low stress  : 60.70
Mean SM participation – High stress : 57.90
N_low = 143, N_high = 143

H1 – Pearson correlation between stress and ΔSM (yearly change)
r = 0.014, p = 0.8174, N = 261


## H2 – Moderating Role of Consumer Confidence

**Hypothesis**

- **H2:** When consumer confidence is low, the effect of economic stress on social-media participation is stronger.
- **H0:** Consumer confidence does not moderate the relationship between stress and participation.

We estimate the following linear regression model:

> sm_participation_{c,t} = β0  
> &nbsp;&nbsp;+ β1 · stress_{c,t}  
> &nbsp;&nbsp;+ β2 · cci_{c,t}  
> &nbsp;&nbsp;+ β3 · (stress_{c,t} × cci_{c,t})  
> &nbsp;&nbsp;+ ε_{c,t}

Where:

- β1 measures the direct effect of economic stress  
- β2 measures the effect of consumer confidence  
- β3 measures whether the effect of stress depends on consumer confidence

In [3]:
# ------------------------------------------------------------
# H2 – OLS regression with interaction: sm_participation ~ stress * cci
# ------------------------------------------------------------

df_h2 = panel_a.dropna(subset=["stress", "cci", "sm_participation"]).copy()

model_h2 = smf.ols("sm_participation ~ stress * cci", data=df_h2).fit()

print(model_h2.summary())

# Extract key coefficients
coef = model_h2.params
pvals = model_h2.pvalues

print("\nKey coefficients:")
print(f"stress      coef = {coef['stress']:.3f}, p = {pvals['stress']:.4f}")
print(f"cci         coef = {coef['cci']:.3f}, p = {pvals['cci']:.4f}")
print(f"stress:cci  coef = {coef['stress:cci']:.3f}, p = {pvals['stress:cci']:.4f}")

                            OLS Regression Results                            
Dep. Variable:       sm_participation   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     3.027
Date:                Sun, 30 Nov 2025   Prob (F-statistic):             0.0299
Time:                        23:58:50   Log-Likelihood:                -1103.7
No. Observations:                 286   AIC:                             2215.
Df Residuals:                     282   BIC:                             2230.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    231.1110     57.811      3.998      0.0

## H3 – Platform Shifts Under Economic Stress

**Hypothesis**

- **H3-1:** Increases in unemployment are associated with increases in professional-platform share (e.g., LinkedIn).
- **H3-2:** Increases in inflation are associated with increases in entertainment-platform share (e.g., Facebook, Instagram, YouTube).
- **H0:** Short-run changes in unemployment and inflation are not systematically related to changes in platform categories.

We use the monthly panel `panel_monthly_h3.csv` and focus on month-to-month changes within each country:

- `d_unemp` – monthly change in unemployment rate  
- `d_infl` – monthly change in HICP index  
- `d_prof` – monthly change in professional platform share  
- `d_ent` – monthly change in entertainment platform share  

For each pair we compute a Pearson correlation after dropping rows with missing values.

In [4]:
# ------------------------------------------------------------
# H3 – Pearson correlations for monthly changes
# ------------------------------------------------------------

def safe_corr(df, x, y, label):
    # Compute Pearson r between x and y, dropping rows with missing values.
    sub = df[[x, y]].dropna()
    n = len(sub)
    if n < 2:
        print(f"{label}: not enough overlapping non-missing observations (n={n}).")
        return np.nan, np.nan, n
    r, p = st.pearsonr(sub[x], sub[y])
    print(f"{label}: r = {r:.3f}, p = {p:.3g}, n = {n}")
    return r, p, n

print("Monthly panel loaded for H3.")
print("Columns:", panel_m.columns.tolist())
print("Shape  :", panel_m.shape)

print("\n==============================")
print("H3-1: ΔUnemployment → ΔProfessional share")
print("==============================")
r1, p1, n1 = safe_corr(panel_m, "d_unemp", "d_prof",
                       "H3-1 ΔUnemployment vs ΔProfessional share")

print("\n==============================")
print("H3-2: ΔInflation → ΔEntertainment share")
print("==============================")
r2, p2, n2 = safe_corr(panel_m, "d_infl", "d_ent",
                       "H3-2 ΔInflation vs ΔEntertainment share")

print("\nSummary:")
print(f"H3-1: r = {r1}, p = {p1}, n = {n1}")
print(f"H3-2: r = {r2}, p = {p2}, n = {n2}")

Monthly panel loaded for H3.
Columns: ['geo', 'year', 'month', 'facebook', 'instagram', 'youtube', 'linkedin', 'unemployment', 'hicp_index', 'ent_share', 'prof_share', 'd_unemp', 'd_infl', 'd_ent', 'd_prof']
Shape  : (306593, 15)

H3-1: ΔUnemployment → ΔProfessional share
H3-1 ΔUnemployment vs ΔProfessional share: r = -0.000, p = 0.801, n = 286254

H3-2: ΔInflation → ΔEntertainment share
H3-2 ΔInflation vs ΔEntertainment share: r = 0.054, p = 5.71e-178, n = 278147

Summary:
H3-1: r = -0.00047219379527208786, p = 0.8005499805737276, n = 286254
H3-2: r = 0.0538947131659661, p = 5.711749084964951e-178, n = 278147
